# 08 - EdDSA и подписи Шнорра (CRYPTO-12)

Этот notebook покрывает EdDSA (Ed25519), подписи Schnorr и их сравнение с ECDSA.

**Что вы изучите:**
1. EdDSA подпись и верификация с библиотекой ecdsa (Ed25519)
2. Детерминированный nonce -- ключевое свойство EdDSA
3. Schnorr подпись вручную (образовательная реализация)
4. Агрегация подписей -- уникальное свойство Schnorr
5. Сравнение ECDSA, EdDSA и Schnorr на одном сообщении
6. Упражнения: MuSig, пороговые подписи

**Используемые библиотеки:** `ecdsa`, `hashlib`, стандартная библиотека Python

In [ ]:
import hashlib
import os
from ecdsa import SECP256k1, Ed25519, SigningKey, VerifyingKey
from ecdsa.numbertheory import inverse_mod

print("Библиотеки загружены")
print(f"Ed25519 порядок: {Ed25519.order}")
print(f"Ed25519 размер ключа: {Ed25519.baselen} байт")

---
## Часть 1: EdDSA подпись и верификация (Ed25519)

Ed25519 -- стандартная реализация EdDSA на кривой Curve25519.
Используется в Solana, Polkadot, Cardano, SSH (ssh-ed25519), TLS 1.3.

In [ ]:
# Генерация ключей Ed25519
sk_ed = SigningKey.generate(curve=Ed25519)
vk_ed = sk_ed.get_verifying_key()

print("=== Ed25519 (Solana/Polkadot) ===")
print(f"Приватный ключ: {sk_ed.to_string().hex()}")
print(f"Публичный ключ: {vk_ed.to_string().hex()}")
print(f"Размер приватного ключа: {len(sk_ed.to_string())} байт")
print(f"Размер публичного ключа: {len(vk_ed.to_string())} байт")

# Подпись
message = b"Solana transaction: transfer 10 SOL"
signature = sk_ed.sign(message)

print(f"\nСообщение: {message.decode()}")
print(f"Подпись: {signature.hex()[:32]}...")
print(f"Размер подписи: {len(signature)} байт")

# Верификация
assert vk_ed.verify(signature, message)
print("Подпись верифицирована!")

---
## Часть 2: Детерминированный nonce EdDSA

**Ключевое отличие EdDSA от ECDSA:** nonce вычисляется детерминированно.

В EdDSA: `k = H(private_key_prefix || message)`

Это значит:
- Один и тот же ключ + сообщение = одна и та же подпись
- Нет зависимости от генератора случайных чисел
- Невозможна атака повторного использования nonce

In [ ]:
# Демонстрация: EdDSA всегда дает одинаковую подпись
sk = SigningKey.generate(curve=Ed25519)
vk = sk.get_verifying_key()

msg = b"Deterministic nonce demo"

sigs = [sk.sign(msg) for _ in range(10)]

print("EdDSA подписи одного сообщения:")
for i, sig in enumerate(sigs[:5], 1):
    print(f"  sig{i}: {sig.hex()[:40]}...")

all_same = all(s == sigs[0] for s in sigs)
print(f"\nВсе 10 подписей идентичны: {all_same}")
print("Детерминированный nonce -> одинаковая подпись")

In [ ]:
# Сравнение: EdDSA vs ECDSA (детерминизм)
sk_ecdsa = SigningKey.generate(curve=SECP256k1)
sk_eddsa = SigningKey.generate(curve=Ed25519)

msg = b"Same message for both"

# ECDSA: каждый раз разная подпись
ecdsa_sigs = [sk_ecdsa.sign(msg).hex()[:24] for _ in range(5)]
print("ECDSA (случайный k):")
for i, s in enumerate(ecdsa_sigs, 1):
    print(f"  sig{i}: {s}...")
print(f"  Уникальных: {len(set(ecdsa_sigs))}/5")

# EdDSA: всегда одна подпись
eddsa_sigs = [sk_eddsa.sign(msg).hex()[:24] for _ in range(5)]
print(f"\nEdDSA (детерминированный k):")
for i, s in enumerate(eddsa_sigs, 1):
    print(f"  sig{i}: {s}...")
print(f"  Уникальных: {len(set(eddsa_sigs))}/5")

print(f"\nEdDSA безопаснее: nonce reuse атака невозможна!")

In [ ]:
# Разные сообщения -> разные подписи (разный nonce)
messages = [
    b"Message 1",
    b"Message 2",
    b"Message 3",
    b"Message 1",  # повтор!
]

print("EdDSA: разные сообщения -> разные подписи:")
sigs = []
for msg in messages:
    sig = sk_eddsa.sign(msg)
    sigs.append(sig)
    print(f"  {msg.decode():12s} -> {sig.hex()[:24]}...")

print(f"\nMessage 1 и Message 1 (повтор): подписи {'совпадают' if sigs[0] == sigs[3] else 'различаются'}")
print(f"Message 1 и Message 2: подписи {'совпадают' if sigs[0] == sigs[1] else 'различаются'}")

---
## Часть 3: Schnorr подпись вручную

Реализуем подпись Schnorr на secp256k1 (как в Bitcoin Taproot BIP 340).

**Алгоритм Schnorr:**
- Подпись: R = kG, e = H(R || P || m), s = k + e*d mod n
- Верификация: sG == R + eP

In [ ]:
# Schnorr подпись на secp256k1 (образовательная реализация)

G = SECP256k1.generator
n = SECP256k1.order

# Генерация ключей
d = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
P = d * G

print(f"Приватный ключ d: {hex(d)[:20]}...")
print(f"Публичный ключ P.x: {hex(P.x())[:20]}...")

# Сообщение
message = b"Schnorr signature demo for Bitcoin Taproot"

# === ПОДПИСЬ ===

# Шаг 1: Случайный nonce k
k = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
R = k * G
print(f"\n--- Подпись ---")
print(f"Шаг 1: k (nonce), R = k*G")
print(f"  R.x = {hex(R.x())[:20]}...")

# Шаг 2: e = H(R.x || P.x || message)
# В BIP 340 используется tagged hash, здесь упрощаем до SHA-256
e_input = R.x().to_bytes(32, 'big') + P.x().to_bytes(32, 'big') + message
e = int(hashlib.sha256(e_input).hexdigest(), 16) % n
print(f"Шаг 2: e = H(R || P || m) = {hex(e)[:20]}...")

# Шаг 3: s = k + e*d mod n  (ПРОСТО! Сравните с ECDSA: s = k^(-1)(h + r*d))
s = (k + e * d) % n
print(f"Шаг 3: s = k + e*d mod n = {hex(s)[:20]}...")

print(f"\nПодпись Schnorr: (R, s)")
print(f"  R.x = {hex(R.x())[:20]}...")
print(f"  s   = {hex(s)[:20]}...")

In [ ]:
# === ВЕРИФИКАЦИЯ Schnorr ===

# Дано: подпись (R, s), публичный ключ P, сообщение m

# Шаг 1: Вычисляем e = H(R.x || P.x || m)
e_verify = int(hashlib.sha256(
    R.x().to_bytes(32, 'big') + P.x().to_bytes(32, 'big') + message
).hexdigest(), 16) % n

# Шаг 2: Проверяем s*G == R + e*P
lhs = s * G        # s * G
rhs = R + e_verify * P  # R + e * P

valid = (lhs.x() == rhs.x() and lhs.y() == rhs.y())
print(f"Верификация Schnorr:")
print(f"  s*G    = ({hex(lhs.x())[:20]}..., ...)")
print(f"  R + eP = ({hex(rhs.x())[:20]}..., ...)")
print(f"\n  s*G == R + eP: {valid}")

print(f"\nПочему работает:")
print(f"  s*G = (k + e*d)*G = k*G + e*d*G = R + e*P")

---
## Часть 4: Агрегация подписей Schnorr

Ключевое свойство Schnorr: подписи можно **складывать**!

Если Алиса (d1, P1) и Боб (d2, P2) подписывают одно сообщение:
- s1 = k1 + e*d1
- s2 = k2 + e*d2
- s_agg = s1 + s2 = (k1+k2) + e*(d1+d2)
- Верификация: s_agg * G == R_agg + e * P_agg

**Это невозможно в ECDSA из-за k^(-1) в формуле!**

In [ ]:
# Агрегация подписей Schnorr (упрощенная демонстрация)

# Алиса
d1 = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
P1 = d1 * G
k1 = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
R1 = k1 * G

# Боб
d2 = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
P2 = d2 * G
k2 = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
R2 = k2 * G

print("Алиса и Боб подписывают совместно:")
print(f"  P1 (Алиса): {hex(P1.x())[:16]}...")
print(f"  P2 (Боб):   {hex(P2.x())[:16]}...")

# Агрегированные значения
P_agg = P1 + P2  # Не точка (d1+d2)*G
R_agg = R1 + R2  # (k1+k2)*G

print(f"\n  P_agg = P1 + P2: {hex(P_agg.x())[:16]}...")
print(f"  R_agg = R1 + R2: {hex(R_agg.x())[:16]}...")

# Вычисляем e для агрегированных значений
message = b"Joint transaction: Alice and Bob agree"
e_agg = int(hashlib.sha256(
    R_agg.x().to_bytes(32, 'big') + P_agg.x().to_bytes(32, 'big') + message
).hexdigest(), 16) % n

# Каждый вычисляет свою часть подписи
s1 = (k1 + e_agg * d1) % n
s2 = (k2 + e_agg * d2) % n

# Агрегированная подпись
s_agg = (s1 + s2) % n

print(f"\nЧастичные подписи:")
print(f"  s1 (Алиса) = {hex(s1)[:16]}...")
print(f"  s2 (Боб)   = {hex(s2)[:16]}...")
print(f"  s_agg      = {hex(s_agg)[:16]}...")

In [ ]:
# Верификация агрегированной подписи

# Верификатор видит только: P_agg, R_agg, s_agg, message
# Он НЕ знает P1, P2, s1, s2 по отдельности!

lhs_agg = s_agg * G
rhs_agg = R_agg + e_agg * P_agg

valid_agg = (lhs_agg.x() == rhs_agg.x() and lhs_agg.y() == rhs_agg.y())

print(f"Верификация агрегированной подписи:")
print(f"  s_agg * G == R_agg + e * P_agg: {valid_agg}")
print(f"\nПочему работает:")
print(f"  s_agg * G = (s1 + s2) * G")
print(f"            = (k1 + e*d1 + k2 + e*d2) * G")
print(f"            = (k1 + k2)*G + e*(d1 + d2)*G")
print(f"            = R_agg + e * P_agg")
print(f"\nАгрегированная подпись выглядит как обычная одиночная подпись!")
print(f"Верификатор не может отличить мультиподпись от обычной.")
print(f"Это основа Bitcoin Taproot (MuSig2).")

---
## Часть 5: Сравнение ECDSA vs EdDSA vs Schnorr

Подписываем одно сообщение тремя алгоритмами и сравниваем.

In [ ]:
import time

message = b"Compare ECDSA, EdDSA, and Schnorr signatures"
N = 100

# === ECDSA (secp256k1) ===
sk_ecdsa = SigningKey.generate(curve=SECP256k1)
vk_ecdsa = sk_ecdsa.get_verifying_key()

start = time.time()
for _ in range(N):
    sig_ecdsa = sk_ecdsa.sign(message)
ecdsa_sign_time = (time.time() - start) / N

start = time.time()
for _ in range(N):
    vk_ecdsa.verify(sig_ecdsa, message)
ecdsa_verify_time = (time.time() - start) / N

# === EdDSA (Ed25519) ===
sk_eddsa = SigningKey.generate(curve=Ed25519)
vk_eddsa = sk_eddsa.get_verifying_key()

start = time.time()
for _ in range(N):
    sig_eddsa = sk_eddsa.sign(message)
eddsa_sign_time = (time.time() - start) / N

start = time.time()
for _ in range(N):
    vk_eddsa.verify(sig_eddsa, message)
eddsa_verify_time = (time.time() - start) / N

# === Сравнение ===
print(f"{'':20s} {'ECDSA':>12s} {'EdDSA':>12s}")
print(f"{'-'*46}")
print(f"{'Кривая':20s} {'secp256k1':>12s} {'Ed25519':>12s}")
print(f"{'Размер подписи':20s} {f'{len(sig_ecdsa)} байт':>12s} {f'{len(sig_eddsa)} байт':>12s}")
print(f"{'Подпись':20s} {f'{ecdsa_sign_time*1000:.2f} мс':>12s} {f'{eddsa_sign_time*1000:.2f} мс':>12s}")
print(f"{'Верификация':20s} {f'{ecdsa_verify_time*1000:.2f} мс':>12s} {f'{eddsa_verify_time*1000:.2f} мс':>12s}")
print(f"{'Детерминизм':20s} {'Нет (*)':>12s} {'Да':>12s}")
print(f"{'Агрегация':20s} {'Нет':>12s} {'Нет (**)':>12s}")
print(f"\n(*) ECDSA может быть детерминированной с RFC 6979 (sign_deterministic)")
print(f"(**) Schnorr на Ed25519 поддерживает агрегацию (sr25519 в Polkadot)")

In [ ]:
# Проверка: ECDSA подписи всегда разные, EdDSA -- одинаковые
ecdsa_sigs = [sk_ecdsa.sign(message) for _ in range(5)]
eddsa_sigs = [sk_eddsa.sign(message) for _ in range(5)]

print("ECDSA подписи одного сообщения:")
print(f"  Уникальных: {len(set(s.hex() for s in ecdsa_sigs))}/5 (каждый раз новый k)")

print(f"\nEdDSA подписи одного сообщения:")
print(f"  Уникальных: {len(set(s.hex() for s in eddsa_sigs))}/5 (детерминированный k)")

# ECDSA с RFC 6979 тоже детерминированная
ecdsa_det_sigs = [sk_ecdsa.sign_deterministic(message) for _ in range(5)]
print(f"\nECDSA RFC 6979 (sign_deterministic):")
print(f"  Уникальных: {len(set(s.hex() for s in ecdsa_det_sigs))}/5")

---
## Упражнения

### Упражнение 1: Базовый MuSig

Реализуйте простой 2-of-2 MuSig протокол:

1. Алиса и Боб генерируют ключевые пары (d1, P1) и (d2, P2)
2. Агрегированный ключ: P_agg = P1 + P2
3. Каждый выбирает nonce: R1, R2. R_agg = R1 + R2
4. e = H(R_agg || P_agg || message)
5. s1 = k1 + e*d1, s2 = k2 + e*d2
6. s_agg = s1 + s2
7. Верифицируйте: s_agg * G == R_agg + e * P_agg

In [ ]:
# Упражнение 1: ваш код здесь

# G = SECP256k1.generator
# n = SECP256k1.order
# 
# # Алиса
# d1 = ...
# P1 = d1 * G
# k1 = ...
# R1 = k1 * G
# 
# # Боб
# d2 = ...
# P2 = d2 * G
# k2 = ...
# R2 = k2 * G
# 
# # Агрегация и подпись
# P_agg = ...
# R_agg = ...
# e = ...
# s_agg = ...
# 
# # Верификация
# assert (s_agg * G).x() == (R_agg + e * P_agg).x()

### Упражнение 2: Schnorr верификация с неправильным ключом

Покажите, что подпись Schnorr нельзя верифицировать с чужим публичным ключом:
1. Алиса подписывает сообщение
2. Попробуйте верифицировать с ключом Боба
3. Объясните, почему верификация провалится

In [ ]:
# Упражнение 2: ваш код здесь

# d_alice = ...
# P_alice = d_alice * G
# d_bob = ...
# P_bob = d_bob * G
# 
# # Алиса подписывает
# k = ...
# R = k * G
# e_alice = H(R || P_alice || msg)
# s_alice = k + e_alice * d_alice
# 
# # Верификация с P_alice (должна пройти)
# ...
# 
# # Верификация с P_bob (должна провалиться)
# e_bob = H(R || P_bob || msg)  # e будет другой!
# ...

---
## Челлендж: Простая пороговая подпись (2-of-3)

Реализуйте упрощенную пороговую подпись, где любые 2 из 3 участников
могут создать валидную подпись.

**Идея:** используйте полиномы Шамира для разделения секрета d на 3 доли (shares),
где любые 2 доли позволяют восстановить d.

1. Выберите секрет d и случайный коэффициент a
2. Полином: f(x) = d + a*x mod n
3. Доли: s1 = f(1), s2 = f(2), s3 = f(3)
4. Любые 2 доли восстанавливают d через интерполяцию Лагранжа
5. Используйте восстановленный d для Schnorr подписи

In [ ]:
# Челлендж: Пороговая подпись

# def shamir_split(secret, n_shares, threshold, modulus):
#     """Разбивает secret на n_shares долей с порогом threshold."""
#     # Генерируем коэффициенты полинома степени threshold-1
#     coeffs = [secret] + [int.from_bytes(os.urandom(32), 'big') % modulus
#                          for _ in range(threshold - 1)]
#     shares = []
#     for i in range(1, n_shares + 1):
#         val = sum(c * pow(i, j, modulus) for j, c in enumerate(coeffs)) % modulus
#         shares.append((i, val))
#     return shares
# 
# def lagrange_interpolate(shares, x, modulus):
#     """Интерполяция Лагранжа в точке x."""
#     result = 0
#     for i, (xi, yi) in enumerate(shares):
#         num, den = 1, 1
#         for j, (xj, _) in enumerate(shares):
#             if i != j:
#                 num = (num * (x - xj)) % modulus
#                 den = (den * (xi - xj)) % modulus
#         result = (result + yi * num * inverse_mod(den, modulus)) % modulus
#     return result
# 
# # Разделяем секрет на 3 доли с порогом 2
# d_secret = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
# shares = shamir_split(d_secret, 3, 2, n)
# 
# # Восстанавливаем из любых 2 долей
# d_recovered = lagrange_interpolate(shares[:2], 0, n)
# assert d_recovered == d_secret
# 
# # Подписываем Schnorr с восстановленным ключом
# ...